In [ ]:
import sys
import os

if 'root_dir' not in globals():
    # rootディレクトリへのパスを設定
    root_dir = os.path.abspath(os.path.join(os.getcwd(), '../'))
    os.chdir(root_dir)

In [ ]:
import logging
# ロギングの設定
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [ ]:
import pandas as pd

#データベースへ接続するエンジンを作成
from my_codes.database_setting import Engine
from my_codes.database_setting import Base

#データベースのテーブルとマッピングする
from my_codes.notes_database import Notes

from sqlalchemy.orm import sessionmaker
from sqlalchemy import func

#セッションを作成
Session = sessionmaker(bind=Engine)
session = Session()

In [ ]:
# 検索語を指定して，特定の用語を含むノートを検索
search_word = '料理'
result = session.query(Notes.key, Notes.created_at ,Notes.tokenized_body).filter(Notes.tokenized_body.like(f'%{search_word}%'))
session.close()

data = pd.DataFrame(result, columns=['key','create_at', 'tokenized_body'])

In [ ]:
import ast  # For converting string representation of lists to actual lists

# Convert the string representation of lists in 'tokenized_body' to actual lists
data['tokenized_body'] = data['tokenized_body'].apply(ast.literal_eval)

# Display the transformed data to ensure correct conversion
data.head()

In [ ]:
from gensim import corpora, models
# !pip install scipy==1.12
# 最新のscipyのバージョンだとtriuがうまくダウンロードできないので、バージョンを指定してインストールする

# Prepare the list of tokens for gensim
texts = data['tokenized_body'].tolist()

# Create a dictionary representation of the documents
# 各単語にユニークなIDを割り当てる
dictionary = corpora.Dictionary(texts)

# Filter out words that occur less than 20 documents, or more than 50% of the documents
dictionary.filter_extremes(no_below=20, no_above=0.5)

In [ ]:
# Convert document into the bag-of-words (BoW) format = list of (token_id, token_count)
corpus = [dictionary.doc2bow(text) for text in texts]

In [ ]:
# Set up the LDA model
num_topics = 20

# LDAモデルを構築
lda_model = models.LdaModel(
    corpus=corpus,  # コーパス、文書のバグ・オブ・ワーズ表現
    num_topics=num_topics,  # 抽出するトピックの数
    id2word=dictionary,  # 単語IDと単語のマッピング
    passes=3,  # トレーニング中にコーパス全体を繰り返す回数
    iterations=50,  # 各パスで各文書内の反復回数
    alpha='auto',  # トピック分布ごとのハイパーパラメータを自動で学習
    eta='auto',  # 単語分布ごとのハイパーパラメータを自動で学習
    random_state=42,  # 再現性のための乱数シード
    chunksize=2000,  # 各トレーニングチャンクで使用される文書の数
    update_every=5  # 各更新のために反復する文書の数
)


In [ ]:
# Show the topics with their terms
topics = lda_model.print_topics(num_words=10)
topics

In [ ]:
from wordcloud import WordCloud

import matplotlib.pyplot as plt
import math

# 日本語フォントのパスを指定
font_path = '/Library/Fonts/Arial Unicode.ttf'

# Calculate number of rows and columns
ncols = math.ceil(math.sqrt(num_topics))
nrows = math.ceil(num_topics / ncols)

# Create a grid of subplots
fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(20, 16))

# Iterate over the topics and plot word clouds in subplots
for i, topic in enumerate(topics):
    # Concatenate the words in the topic
    topic_words = ' '.join(topic[1].split('*'))
    
    # Generate the word cloud
    wordcloud = WordCloud(font_path=font_path, width=400, height=200, colormap='Set2', background_color='white').generate(topic_words)
    
    # Plot the word cloud in the corresponding subplot
    ax = axes[i // 5, i % 5]
    ax.imshow(wordcloud, interpolation='bilinear')
    ax.set_title(f'Topic {topic[0]}')
    ax.axis('off')

# Adjust the spacing between subplots
plt.tight_layout()

#plt.savefig(f'img/word_cloud_{search_word}.png')

# Display the combined image of word clouds
plt.show()




In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns

# 文書ごとのトピック分布を行列に変換
topic_distributions = [lda_model[doc] for doc in corpus]
topic_matrix = np.zeros((len(topic_distributions), num_topics))

for i, dist in enumerate(topic_distributions):
    for topic_id, prob in dist:
        topic_matrix[i, topic_id] = prob

# トピック間の相関行列を計算
topic_correlation_matrix = np.corrcoef(topic_matrix.T)

# DataFrameに変換して可視化
topic_correlation_df = pd.DataFrame(topic_correlation_matrix)
plt.figure(figsize=(10, 8))
sns.heatmap(topic_correlation_df, annot=True, cmap="coolwarm")
plt.title("Topic Correlation Matrix")
plt.show()


In [ ]:
# ドキュメントごとのトピック分布を取得
doc_topic_dist = lda_model.get_document_topics(corpus, minimum_probability=0)

# トピック分布をデータフレームに変換
#doc_topic_dist = pd.DataFrame(doc_topic_dist)

doc_topic_dist = [[topic_prob[1] for topic_prob in doc] for doc in doc_topic_dist]
doc_topic_dist = pd.DataFrame(doc_topic_dist)

# 文書全体におけるトピック分布の合計値を計算
topic_dist_sum =doc_topic_dist.sum(axis=0)

# 可視化
topic_dist_sum.plot(kind='bar', figsize=(10, 6), color='skyblue')

In [ ]:
doc_topic_dist

In [ ]:
# トピックの時系列変化を可視化
# 時系列データとして扱うために、日付をインデックスに設定
data['create_at'] = pd.to_datetime(data['create_at'])

# トピック分布をデータフレームに追加
data = pd.concat([data, doc_topic_dist], axis=1)

# トピック分布の時系列変化を可視化
data.set_index('create_at').drop(columns=['key', 'tokenized_body']).plot(figsize=(12, 8))